In [1]:
# ////////////// <========> Import Libraries <========> ////////////// #

import cv2 as cv
import numpy as np 
from enum import Enum
from skimage.color import rgb2gray
from commonfunctions import *
from skimage.filters import gaussian
from skimage.exposure import adjust_gamma
from haar_features import *
from intagral_image import *
from cartoonization import *
from skimage.transform import resize

# //////// <====> End <====> //////// #

In [2]:
# //////// reads the result of adaboost training and assigns it to list of haar classifiers //////// #
def readClassifiers():
    f = open("classifiersData.txt", "r")
    f = f.readlines()
    features=[]
    f = [line.strip() for line in f]
    for i in range(0,len(f),7):
        feature=HaarLikesFeatures(HaarFeaturesType[f[i]], (int(f[i+1]),int(f[i+2])), int(f[i+3]), int(f[i+4]), int(f[i+5]), int(f[i+6]))
        features.append(feature)
    return features
classifiers = readClassifiers()

In [3]:
# //////// return 1 if image is a face and 0 otherwise //////// #
def imageIsFace(image):
    return 1 if sum([c.vote(image) for c in classifiers[0:10]])  >= 5 and sum([c.vote(image) for c in classifiers[11:20]])  >= 3 and sum([c.vote(image) for c in classifiers[21:30]])  >= 3  else 0

In [4]:
# //////// cartoonize an image in a give range //////// #
def applyCartoonization(frame,x,y,h):
    # frame is the image
    # x is x start for the range
    # y is y start for the range
    # h is the size of the square window
    cv.rectangle(frame,(x,y),(x+h,y+h),(255,0,0),3)
    window = frame[y:y+h, x:x+h, :]
    window = cartoonize(window,21)
    frame[y:y+h,x:x+h,:] = window
    return frame
    

In [5]:
# //////// Loops over an image and extract square windows of different sizes [80,100,...200] //////// #
# it returns the position of the window if it is a face
# it returns -1 if it's not a face
def extractWindows(image):
    x = image.shape[0]
    y = image.shape[1]
    for z in range(200,79,-20):
        windowSize=z
    #extractedWindows=[]
        for i in range(0,x-windowSize,20):
            for j in range(0,y-windowSize,20):
                window = image[i:i+windowSize, j:j+windowSize]
                window = resize(window, (25,25))
                window =window/ window.max()
                integralImg = integral_image(window)
                #print(i,j)
                isFace = imageIsFace(integralImg)
                if isFace == 1:
                    return j,i,windowSize
                    #extractedWindows.append(window)
    return -1,-1,-1

In [6]:
# //// ---> Testing for a single image <--- //// #

frame = cv.imread('test2.jpg')
frame=cv.cvtColor(frame, cv.COLOR_BGR2RGB)
# //// ---> Convert Frame To Gray Scale <--- //// #
gray = cv.cvtColor(frame,cv.COLOR_RGB2GRAY)
# --- End --- #
# //// ---> Normalization <--- //// #
gray = gray/gray.max()
# --- End --- #

# --- Gets the window if a face if available --- #
x,y,h=extractWindows(gray)
if (x != -1):
    frame=applyCartoonization(frame, x,y,h)
# //// ---> Show Frame <--- //// #
show_images([frame])

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Real time video

In [9]:
# ////////////// <========> Run Video <========> ////////////// #

cap=cv.VideoCapture(0)
while(True):
    #read Frames
    ret,frame=cap.read()
    if(ret==False):
        print('no frame')
        break
    else:
        frame=cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        # //// ---> Convert Frame To Gray Scale <--- //// #
        gray = cv.cvtColor(frame,cv.COLOR_RGB2GRAY)
        # --- End --- #
         # //// ---> Normalization <--- //// #
        gray = gray/gray.max()
        # --- End --- #
        # //// ---> Apply Gaussian Filter <--- //// #
        newGray = gaussian(gray, sigma=1, output=None, mode='nearest')
        # --- End --- #
        # //// ---> Apply Gamma correction <--- //// #
        newGray = adjust_gamma(newGray,gamma=1.5)
        # --- End --- #
        
        # //// ---> Integral Image <--- //// #
        #gray=integral_image(newGray)     
        # --- End --- #
        x,y,h=extractWindows(newGray)
        if (x != -1):
            frame=applyCartoonization(frame, x,y,h)
        # //// ---> Show Frame <--- //// #
        frame=cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        cv.imshow('frame', frame)     
        # --- End --- #
        
        #exit
        if cv.waitKey(1)& 0xFF==ord('q'):
            break
        
#cap.realse()
cv.destroyAllWindows()
cap.release()      

# //////// <====> End <====> //////// #